In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, Conv2D, MaxPooling2D, BatchNormalization, Reshape, GlobalAveragePooling2D, Dense, Flatten
from tensorflow.keras.models import Model

def InceptionNucleus(input_layer, filters):
    branches = []
    for f in filters:
        x = Conv1D(filters=32, kernel_size=f, padding='same', activation='relu')(input_layer)
        branches.append(x)
    output = tf.keras.layers.concatenate(branches, axis=-1)
    return output

def create_eeg_cnn(input_shape=(178, 1), num_classes=2):
    inputs = Input(shape=input_shape)

    # Initial 1D Convolutional Layers with Inception Nucleus
    x = InceptionNucleus(inputs, filters=[4, 8, 12])

    # Reshape for 2D Convolutions
    x = Reshape((x.shape[1], x.shape[2], 1))(x) # (178, 96, 1) for example

    # 2D Convolutional Layers
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    # Flatten after 2D Convolutions
    x = Flatten()(x)

    # Output Layer
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

model = create_eeg_cnn(input_shape=(178, 1), num_classes=2)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 178, 1)]             0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 178, 32)              160       ['input_1[0][0]']             
                                                                                                  
 conv1d_1 (Conv1D)           (None, 178, 32)              288       ['input_1[0][0]']             
                                                                                                  
 conv1d_2 (Conv1D)           (None, 178, 32)              416       ['input_1[0][0]']             
                                                                                              

In [1]:
# Read the CSV file
import pandas as pd
df = pd.read_csv("data/data.csv")

# Split the first column into three new columns
df[['second_no', 'version', 'pid']] = df.iloc[:, 0].str.split('.', expand=True)

# Extract the numeric parts and replace X and V directly
df['second_no'] = df['second_no'].str.extract('(\d+)').astype(int)  # Replace X with its numeric part
df['version'] = df['version'].str.extract('(\d+)').astype(int)  # Replace V with its numeric part
# Convert the 'num' column to integer for sorting
# Fill None values in 'num' with a default value (e.g., 0)
df['pid'] = df['pid'].fillna(0).astype(int)
# Rename the 'Unnamed: 0' column to 'id'
df.rename(columns={'Unnamed: 0': 'id'}, inplace=True)

# Reorder the columns to move num, X, V between id and X1
df = df[['id', 'pid', 'second_no', 'version', 'X1'] + [col for col in df.columns if col not in ['id', 'pid', 'second_no', 'version', 'X1']]]

# Sort the DataFrame by the 'num' column
df_sorted = df.sort_values(by=['version', 'pid', 'second_no'])

# Display the sorted DataFrame
df_sorted.head(50)

# Find the maximum existing pid
max_pid = df['pid'].max()
# Identify unique versions that are not 1
versions_with_empty_pid = df[df['version'] != 1]['version'].unique()
# Assign a unique pid to each participant and unify the version to 1
for i, version in enumerate(versions_with_empty_pid):
    new_pid = max_pid + i + 1  # Start from max_pid + 1
    df.loc[(df['version'] == version), 'pid'] = new_pid
    df.loc[(df['version'] == version), 'version'] = 1

# Binary classification: 1: Seizure, 0: Non-seizure
# Change y value to 0 if y != 1
df['y'] = df['y'].apply(lambda x: 0 if x != 1 else x)

X = df.drop(columns=['pid', 'second_no', 'version', 'id', 'y'])  # Features
y = df['y']  # Target variable

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score


def evaluate_classification_metrics(y_true, y_pred, y_pred_prob):
    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)
    
    # Calculate precision
    precision = precision_score(y_true, y_pred)
    
    # Calculate recall
    recall = recall_score(y_true, y_pred)
    
    # Calculate F1 score
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    if len(set(y_true)) > 1:
        # Check if ROC-AUC can be calculated (i.e., both classes are present)
        roc_auc = roc_auc_score(y_true, y_pred_prob)
        # Calculate Cohen's Kappa
        kappa = cohen_kappa_score(y_true, y_pred)
    else:
        roc_auc = None  # Not computable, only one class in y_true
        kappa = None
    
    # Calculate metrics for seizure class (y_label=1)
    precision_seizure = precision_score(y_true, y_pred, pos_label=1)
    recall_seizure = recall_score(y_true, y_pred, pos_label=1)
    f1_seizure = f1_score(y_true, y_pred, pos_label=1)
    
    # Calculate metrics for non-seizure class (y_label=0)
    precision_non_seizure = precision_score(y_true, y_pred, pos_label=0)
    recall_non_seizure = recall_score(y_true, y_pred, pos_label=0)
    f1_non_seizure = f1_score(y_true, y_pred, pos_label=0)
    
    print(f'\nSeizure (y=1):')
    print(f'  Precision: {precision_seizure * 100:.2f} %')
    print(f'  Recall: {recall_seizure * 100:.2f} %')
    print(f'  F1 Score: {f1_seizure * 100:.2f} %')
    
    print(f'\nNon-Seizure (y=0):')
    print(f'  Precision: {precision_non_seizure * 100:.2f} %')
    print(f'  Recall: {recall_non_seizure * 100:.2f} %')
    print(f'  F1 Score: {f1_non_seizure * 100:.2f} %')
    
    # Print metrics
    print(f'\nOverall:')
    print(f'  Accuracy: {accuracy * 100:.2f} %')
    print(f'  Precision: {precision * 100:.2f} %')
    print(f'  Recall: {recall * 100:.2f} %')
    print(f'  F1 Score: {f1 * 100:.2f} %')
    if roc_auc is not None:
        print(f'  ROC-AUC: {roc_auc * 100:.2f} %')
    if kappa is not None:
        print(f'  Cohen\'s Kappa: {kappa * 100:.2f} %')

In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

X_reshaped = np.array(X).reshape(-1, 178, 1)

# Split data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)

# Create the model
model = create_eeg_cnn(input_shape=(178, 1), num_classes=2)

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),  # For evaluation during training
    epochs=30,                         # Number of epochs
    batch_size=32,                     # Batch size (adjust based on memory)
    callbacks=[early_stopping]          # Early stopping
)

# After training, evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# Predict class probabilities for the test set
y_pred_prob = model.predict(X_test)

# Convert probabilities to binary class labels (assuming threshold 0.5)
y_pred = (y_pred_prob[:, 1] > 0.5).astype(int)

# Evaluate classification metrics using the provided function
evaluate_classification_metrics(y_test, y_pred, y_pred_prob[:, 1])

print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

Epoch 1/30
288/288 [==============================] - 49s 168ms/step - loss: 9.5476 - accuracy: 0.9237 - val_loss: 3.8512 - val_accuracy: 0.9587
Epoch 2/30
288/288 [==============================] - 48s 167ms/step - loss: 3.2597 - accuracy: 0.9530 - val_loss: 3.3574 - val_accuracy: 0.9574
Epoch 3/30
288/288 [==============================] - 47s 164ms/step - loss: 2.4595 - accuracy: 0.9604 - val_loss: 4.7436 - val_accuracy: 0.9535
Epoch 4/30
288/288 [==============================] - 48s 166ms/step - loss: 1.7028 - accuracy: 0.9699 - val_loss: 2.1359 - val_accuracy: 0.9613
Epoch 5/30
288/288 [==============================] - 48s 167ms/step - loss: 1.2081 - accuracy: 0.9733 - val_loss: 2.2791 - val_accuracy: 0.9343
Epoch 6/30
288/288 [==============================] - 49s 171ms/step - loss: 0.5948 - accuracy: 0.9836 - val_loss: 3.2205 - val_accuracy: 0.9243
Epoch 7/30
288/288 [==============================] - 50s 174ms/step - loss: 0.7680 - accuracy: 0.9825 - val_loss: 2.8094 - val_ac